In [1]:
# Extract JSON from Camden Trees API and store it in mySQL table in UCL database
!pip install pandas
!pip install sodapy

import pandas as pd
from sodapy import Socrata
import requests
import pymysql
import os
import json

#!!!!this is my registered account with Camden COuncil, if prompted a lot, they will lock it !!!
#Define the API path, app token, the registered account username and password
client = Socrata("opendata.camden.gov.uk","5YbHlvnsIgblrrY3kSuIU9WZ0", username = "ucfnmoa@ucl.ac.uk",password = "Archsara13" )

# Increased the limit to cover all the records
# Data returned as JSON from API / converted to Python list of dictionaries by sodapy.
r = client.get("csqp-kdss", limit=50000)

def validate_string(val):
    if val != None:
        if type(val) is int:
            #for x in val:
            #   print(x)
            return str(val).encode('utf-8')
        else:
            return val
        
# connect with UCL database, enter your user, password and database from week 1       
mydb = pymysql.connect(host='dev.spatialdatacapture.org',
user='ucfnmoa',
passwd='qenewoqiki',
db='ucfnmoa')

cursor = mydb.cursor()

# this file is linked to a crontab scheduled every 6 months
# trees_Camden table was created from the last data update
# check if there is a table with the same name and drop it 
cursor.execute("DROP TABLE IF EXISTS trees_Camden")

# Create a table in mySQL that includes the below columns
# Indicate data types
cursor.execute("CREATE TABLE trees_Camden (No_Trees VARCHAR(255), Site_Name VARCHAR(255),Sci_Name VARCHAR(255), Com_Name VARCHAR(255) ,Inspection_Date VARCHAR(255),Height_M VARCHAR(255), Spread_M VARCHAR(255),Diameter_CM VARCHAR(255),Maturity VARCHAR(255), trees_Con VARCHAR(255), Removed_Soon VARCHAR(255),Newly_Planted VARCHAR(255), Amenity_Value VARCHAR(255),Carbon_StorageKG VARCHAR(255), Carbon_Sequest_YR VARCHAR(255), Pollution_Year_grams VARCHAR(255), Ward_Code VARCHAR(255), Ward_Name VARCHAR(255) , lat DOUBLE , lon DOUBLE,Uploaded VARCHAR(255))")

# For each column, validate string
for i,item in enumerate(r):
    number_of_trees = validate_string(item.get("number_of_trees", None))
    site_name=validate_string(item.get("site_name", None))
    scientific_name = validate_string(item.get("scientific_name", None))
    common_name=validate_string(item.get("common_name", None))
    inspection_date=validate_string(item.get("inspection_date", None))
    height_in_metres = validate_string(item.get("height_in_metres", None))
    spread_in_metres=validate_string(item.get("spread_in_metres", None))
    diameter_in_centimetres_at_breast_height = validate_string(item.get("diameter_in_centimetres_at_breast_height", None))
    maturity=validate_string(item.get("maturity", None))
    physiological_condition = validate_string(item.get("physiological_condition", None))
    tree_set_to_be_removed=validate_string(item.get("tree_set_to_be_removed", None))
    newly_planted = validate_string(item.get("newly_planted", None))
    capital_asset_value_for_amenity_trees= validate_string(item.get("capital_asset_value_for_amenity_trees", None))
    carbon_storage_in_kilograms=validate_string(item.get("carbon_storage_in_kilograms", None))
    gross_carbon_sequestration_per_year_in_kilograms = validate_string(item.get("gross_carbon_sequestration_per_year_in_kilograms", None))
    pollution_removal_per_year_in_grams=validate_string(item.get("pollution_removal_per_year_in_grams", None))
    ward_code = validate_string(item.get("ward_code", None))
    ward_name=validate_string(item.get("ward_name", None))
    latitude =validate_string(item.get("latitude", None))
    longitude = validate_string(item.get("longitude", None))
    last_uploaded = validate_string(item.get("last_uploaded", None))

    # Insert the updated data into the table 
    # This is prompted by the crontab added in putty and linked to this file, it is set to update the data every 6 months
    cursor.execute("INSERT INTO trees_Camden (No_Trees, Site_Name ,Sci_Name , Com_Name  ,Inspection_Date ,Height_M, Spread_M ,Diameter_CM ,Maturity , trees_Con , Removed_Soon ,Newly_Planted, Amenity_Value ,Carbon_StorageKG , Carbon_Sequest_YR, Pollution_Year_grams , Ward_Code, Ward_Name , lat  , lon ,Uploaded) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (number_of_trees , site_name ,scientific_name ,common_name, inspection_date, height_in_metres, spread_in_metres, diameter_in_centimetres_at_breast_height, maturity, physiological_condition , tree_set_to_be_removed, newly_planted , capital_asset_value_for_amenity_trees, carbon_storage_in_kilograms, gross_carbon_sequestration_per_year_in_kilograms ,pollution_removal_per_year_in_grams, ward_code ,ward_name, latitude, longitude,last_uploaded))
    
# Close the connection to the database.
mydb.commit()
cursor.close()

# Print Done to inform the user that the update is finished
print("Done")

#check in mySQL your database, you can find the trees in camden table with all the records!

Done
